In [ ]:
import torch
import warnings

In [ ]:
# PyTorch and general Python warnings
warnings.filterwarnings("ignore")

In [ ]:
import pickle

def load_from_pkl(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

# Load lists from .pkl files
anomaly_sequences = load_from_pkl('anomaly_train.pkl')
normal_sequences = load_from_pkl('normal_train.pkl')

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
class Globalselfatt(nn.Module):
    def __init__(self, out_channels):
        super(Globalselfatt, self).__init__()
        self.out_channels = out_channels

        # Define convolution layers
        self.theta = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=1)
        self.phi = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=1)
        self.b = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=1)

    def forward(self, x):
        # Compute theta, phi, and b
        theta = self.theta(x)
        phi = self.phi(x)
        b = self.b(x)

        # Transpose theta
        theta = theta.permute(0, 2, 1)

        # Compute the attention map using matrix multiplication and softmax
        attention = F.softmax(torch.matmul(theta, phi), dim=-1)

        # Compute the output using the attention map
        output = torch.matmul(b, attention)

        return output

In [ ]:
class HTAMModule(nn.Module):
    def __init__(self, input_channels, hidden_size):
        super(HTAMModule, self).__init__()
        
        # Conv1D layer
        self.conv1d = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3)
        self.relu = nn.ReLU()
        

        self.nln_block = Globalselfatt(out_channels=64)

        # LSTM layer
        self.lstm = nn.LSTM(input_size=64*1022, hidden_size=hidden_size, batch_first=True, bidirectional=False)       #, batch_first=True

    def forward(self, frame_features):
        #print("frame",frame_features.shape)
        
        # Apply Conv1D layer
        conv1d_features = self.conv1d(frame_features)
        #print("conv1",conv1d_features.shape)

        conv1d_features = self.relu(conv1d_features)
        #print("conv1",conv1d_features.shape)
        
        # Apply Globalselfatt block
        nln_features = self.nln_block(conv1d_features)
        #print("nln",nln_features.shape)

        flattened_nln_features = nln_features.reshape(10, -1)
        #print("flat nln", flattened_nln_features.shape)
        
        # Apply LSTM layer
        lstm_features, _ = self.lstm(flattened_nln_features)
        
        return lstm_features

In [ ]:
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2,hidden_size3,  output_size):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_size, hidden_size)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

        self.lyrs = [self.fc]

        self.weight_init()

    def weight_init(self):
        for lyr in self.lyrs:
            torch.nn.init.xavier_uniform_(lyr.weight)

    def forward(self, x):
        x = self.relu( self.fc(x) )
        x = nn.Dropout(0.2)(x)
        x = self.sigmoid(x) 
        return x

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2, hidden_size3, output_size, htam_input_channels, htam_hidden_size):
        super(CombinedModel, self).__init__()

        # HTAM Module
        self.htam_module = HTAMModule(input_channels=htam_input_channels, hidden_size=htam_hidden_size)

        # Classifier Module
        self.classifier = Classifier(input_size, hidden_size, hidden_size2, hidden_size3, output_size)

    def forward(self, x):
        # Pass input through the HTAM Module
        htam_output = self.htam_module(x)

        # Pass the HTAM output through the Classifier
        classifier_output = self.classifier(torch.flatten(htam_output))

        return classifier_output

In [ ]:
import torch.optim as optim
input_size = 16*512
hidden_size = input_size // 2
hidden_size2 = 512
hidden_size3 = 32
output_size = 1

htam_input_channels = 49  
htam_hidden_size = 512    

num_epochs = 200
learning_rate = 0.33

In [ ]:
model = CombinedModel(input_size, hidden_size, hidden_size2, hidden_size3, output_size, htam_input_channels, htam_hidden_size)
model

In [ ]:
batch_size = 64

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()
test_inference = False

history = []

for epoch in range(num_epochs):
    model.train() 
    optimizer.zero_grad()
    batch_loss = 0

    for i in range(0, len(anomaly_sequences), batch_size):
        
        # Anomaly sequences
        for j in range(i, min(i+batch_size, len(anomaly_sequences))):
            #print(f"anomaly {j}")
            fname =  f"swin_embeddings/train/1/1_{j}.pt"
            try:
                abnormal_spatial_embeddings = torch.load(fname)
            except:
                break
            anomaly_output = model(abnormal_spatial_embeddings)
            anomaly_label = torch.ones((1), dtype=torch.float32)
            anomaly_loss = criterion(anomaly_output, anomaly_label)
            batch_loss += anomaly_loss

    for i in range(0, len(normal_sequences), batch_size):

        # Normal sequences
        for j in range(i, min(i+batch_size, len(normal_sequences))):
            #print(f"normal {j}")
            fname =  f"swin_embeddings/train/0/0_{j}.pt"
            try:
                normal_spatial_embeddings = torch.load(fname)
            except:
                break
            normal_output = model(normal_spatial_embeddings)
            normal_label = torch.zeros((1), dtype=torch.float32)
            normal_loss = criterion(normal_output, normal_label)
            batch_loss += normal_loss

    batch_loss.backward()
    optimizer.step()

    if test_inference:
        precision, recall, fpr, tpr, auc, accuracy = inference(model)
        history.append([batch_loss.item(), precision, recall, fpr, tpr, auc])
        print(f'Epoch {epoch+1}, Train Loss: {batch_loss.item()}, Accuracy:{accuracy}, Precision: {precision}, Recall: {recall}, AUC: {auc}')
    else:
        history.append([batch_loss.item()])
        print(f'Epoch {epoch+1}, Train Loss: {batch_loss.item()}')

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, accuracy_score
import numpy as np

def inference(model):
    model.eval()  

    anomaly_test_data = load_from_pkl('anomaly_test.pkl')
    normal_test_data = load_from_pkl('normal_test.pkl')

    all_predictions = []
    all_labels = []

    with torch.no_grad():
        # For anomaly data
        for i in range(len(anomaly_test_data)):
            fname =  f"swin_embeddings/test/1/1_{i}.pt"
            data = torch.load(fname)
            output = model(data)
            prediction = (output > 0.5).float().item()
            all_predictions.append(prediction)
            all_labels.append(1.0)  

        # For normal data
        for i in range(len(normal_test_data)):
            fname =  f"swin_embeddings/test/0/0_{i}.pt"
            data = torch.load(fname)
            output = model(data)
            prediction = (output > 0.5).float().item()
            all_predictions.append(prediction)
            all_labels.append(0.0)  

    # Calculate metrics
    #print(all_labels)
    #print(all_predictions)
    #print(len(all_labels))
    labels = "Test/GroundTruth"
    pred = "Test/predictedValues"
    labels_roc = "Test/predictedLabels"
    pred_roc = "Test/PredictedROC"
    print(len(labels))
    precision = precision_score(all_labels, pred)
    recall = recall_score(all_labels, pred)
    accu = accuracy_score(all_labels, pred)
    
    
    all_scores = [output for output in fake_pred_roc]
    fpr, tpr, thresholds = roc_curve(fake_labels_roc, all_scores)
    auc = roc_auc_score(fake_labels_roc, all_scores)

    return precision, recall, fpr, tpr, auc, accu   # Return the metrics

In [ ]:
batch_size = 64

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()
test_inference = False

history = []

for epoch in range(num_epochs):
    model.train() 
    optimizer.zero_grad()
    batch_loss = 0

    for i in range(0, len(anomaly_sequences), batch_size):
        
        # Anomaly sequences
        for j in range(i, min(i+batch_size, len(anomaly_sequences))):
            #print(f"anomaly {j}")
            fname =  f"swin_embeddings/train/1/1_{j}.pt"
            try:
                abnormal_spatial_embeddings = torch.load(fname)
            except:
                break
            anomaly_output = model(abnormal_spatial_embeddings)
            anomaly_label = torch.ones((1), dtype=torch.float32)
            anomaly_loss = criterion(anomaly_output, anomaly_label)
            batch_loss += anomaly_loss

    for i in range(0, len(normal_sequences), batch_size):

        # Normal sequences
        for j in range(i, min(i+batch_size, len(normal_sequences))):
            #print(f"normal {j}")
            fname =  f"swin_embeddings/train/0/0_{j}.pt"
            try:
                normal_spatial_embeddings = torch.load(fname)
            except:
                break
            normal_output = model(normal_spatial_embeddings)
            normal_label = torch.zeros((1), dtype=torch.float32)
            normal_loss = criterion(normal_output, normal_label)
            batch_loss += normal_loss

    batch_loss.backward()
    optimizer.step()

    if test_inference:
        precision, recall, fpr, tpr, auc, accuracy = inference(model)
        history.append([batch_loss.item(), precision, recall, fpr, tpr, auc])
        print(f'Epoch {epoch+1}, Train Loss: {batch_loss.item()}, Accuracy:{accuracy}, Precision: {precision}, Recall: {recall}, AUC: {auc}')
    else:
        history.append([batch_loss.item()])
        print(f'Epoch {epoch+1}, Train Loss: {batch_loss.item()}')